In [1]:
from keras.callbacks import TensorBoard
%run data_gen.ipynb
%run models.ipynb

c:\users\amany\appdata\local\conda\conda\envs\gputf3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
dg = DataGen()

n_features = len(char2int)
n_steps_in = dg.max_root_len
n_steps_out = dg.max_output_len


In [3]:
batch_size = 128
n_batches = int(len(dg.words) * .7 / batch_size) 
gen = dg.gen_rnn(batch_size=batch_size, n_batches=n_batches)
[x1, x2, x3], y = next(gen)
print(x1.shape, x2.shape, x3.shape, y.shape)

(128, 15, 28) (128, 31, 28) (128, 32) (128, 31, 28)


In [12]:
# define model 
# train, infenc, infdec = seq2(n_features, n_features, 64, dg.word_feat_len)
train, infenc, infdec = rnn_model(n_features, n_features, dg.word_feat_len, 64, 64)
train.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
train.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
root_word_input (InputLayer)    (None, None, 28)     0                                            
__________________________________________________________________________________________________
word_feature_input (InputLayer) (None, 32)           0                                            
__________________________________________________________________________________________________
encoder_gru (GRU)               [(None, 64), (None,  17856       root_word_input[0][0]            
__________________________________________________________________________________________________
feature_output (Dense)          (None, 5)            165         word_feature_input[0][0]         
__________________________________________________________________________________________________
concatenat

In [5]:
history = train.fit_generator(gen, steps_per_epoch=n_batches, epochs = 2)

Epoch 1/2
5702/5702 [==============================] - 892s 156ms/step - loss: 0.1533 - acc: 0.9506
Epoch 2/2
5702/5702 [==============================] - 892s 157ms/step - loss: 0.0123 - acc: 0.9933


In [6]:
test_n_batches, test_batch_size = 30, 100 
test_gen = dg.gen2(batch_size=test_batch_size, n_batches=test_n_batches, trainset=False)

In [7]:
total, correct = 0, 0
in_word = 0
sims = []
for b in range(test_n_batches):
    [X1, X2, X3], y = next(test_gen)
    for j in range(test_batch_size):
        X33 = X3[j].reshape((1, X3.shape[1])) 
        X11 = X1[j].reshape((1, X1.shape[1], X1.shape[2]))
        target = predict2(infenc, infdec, X11, X33, n_steps_out, n_features)
        root = ''.join(dg.one_hot_decode(X1[j]))#.replace('&', ' ')
        word = ''.join(dg.one_hot_decode(y[j]))#.replace('&', ' ')
        targetS = ''.join(dg.one_hot_decode(target))#.replace('&', ' ')
        sims.append(dg.word_sim(word, targetS))
        if dg.one_hot_decode(y[j]) == dg.one_hot_decode(target):
            correct += 1
        if root.strip() in targetS.strip():
            in_word += 1
    print(b, root, word, targetS)
    total += test_batch_size
    #37.83
    #68%
    #77.33
    #79.37%
word_sim_average = sum(sims)/len(sims)
print('Word Similarity Average: {0:.2f}%'.format(word_sim_average))
print('Exact Accuracy: %.2f%%' % (float(correct)/float(total)*100.0))
print('Word in Accuracy: %.2f%%' % (float(in_word)/float(total)*100.0))

0 matay           matayiyogaa&                    matayiyogaa&                   
1 sal             salissiyaro&                    salissiyaro&                   
2 maaret          maaretaro&                      maaretaro&                     
3 yedd            yeddaanaagoo&                   yeddaanaagoo&                  
4 cagg            cagissidori&                    cagissidori&                   
5 achch           achchissiyonaa&                 achchissiyonaa&                
6 homppat         homppatiyonii&                  homppatiyo&                    
7 paall           paallogeetoo&                   paallidogeetoo&                
8 baxx            baxissennee&                    baxissennee&                   
9 warqqall        warqqalliis&                    warqqalliis&                   
10 daadir          daadira&                        daadira&                       
11 paatt           paatissibeokkonaa&              paatissibeokkonaa&             
12 qoom       